In [19]:
import os
import requests
import aiohttp
import asyncio
import nest_asyncio
import pandas as pd
import time
import scrapy
from scrapy_playwright.page import PageMethod
from bs4 import BeautifulSoup
import nest_asyncio
import glob
import numpy as np
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz, process
import re
import random
import textwrap
from copy import deepcopy
from apify_client import ApifyClient
from urllib.error import HTTPError
from requests.exceptions import ConnectionError, RequestException


In [20]:
parquet_path = "./Staging/Bronze/bronze_data_1.parquet"
if os.path.exists(parquet_path):
    acra_data_filtered_by_industry = pd.read_parquet(parquet_path, engine="fastparquet")
    print(f"Loaded {len(acra_data_filtered_by_industry)} rows from {parquet_path}")
    print(acra_data_filtered_by_industry.shape)
else:
    raise FileNotFoundError(f"Parquet file not found at {parquet_path}")

Loaded 10 rows from ./Staging/Bronze/bronze_data_1.parquet
(10, 14)


In [21]:
acra_data_filtered_by_industry

,UEN,ENTITY_NAME,BUSINESS_CONSTITUTION_DESCRIPTION,ENTITY_TYPE_DESCRIPTION,ENTITY_STATUS_DESCRIPTION,REGISTRATION_INCORPORATION_DATE,PRIMARY_SSIC_CODE,SECONDARY_SSIC_CODE,STREET_NAME,POSTAL_CODE,PARENT_INDUSTRY,INDUSTRY_TYPE,SUB_INDUSTRY,DESCRIPTION
0,53431824W,TUTORSVILLE.SG,SOLE-PROPRIETOR,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,07-04-2021,85509,na,COMPASSVALE WALK,540230,Others,Educational,Tuition & Enrichment Centers,Training Courses N.E.C.
1,202344030R,CHEM AFFINITY LEARNING CENTRE PTE. LTD.,None,LOCAL COMPANY,LIVE COMPANY,04-11-2023,85509,na,BEACH ROAD,189695,Others,Educational,Tuition & Enrichment Centers,Training Courses N.E.C.
2,T15LL1885G,EDUREACH SERVICES LLP,None,LIMITED LIABILITY PARTNERSHIP,LIVE,11-11-2015,85509,74901,TAMPINES STREET 23,527201,Others,Educational,Tuition & Enrichment Centers,Training Courses N.E.C.
3,53200915X,THINK ARTS,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,06-10-2011,85509,na,YARROW GARDENS,455021,Others,Educational,Tuition & Enrichment Centers,Training Courses N.E.C.
4,201733719E,JUS INFANTS @ MACPHERSON PTE. LTD.,None,LOCAL COMPANY,LIVE COMPANY,22-11-2017,88911,na,KALLANG PUDDING ROAD,349318,Others,Hospital,Social Services (Without Accommodations),Infant Care Services; Child Minding Services F...
5,53227394W,MATHS TABLET,SOLE-PROPRIETOR,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,04-12-2012,85509,na,ANG MO KIO AVENUE 10,560555,Others,Educational,Tuition & Enrichment Centers,Training Courses N.E.C.
6,202209857Z,YORK EDUCATION PTE. LTD.,None,LOCAL COMPANY,LIVE COMPANY,22-03-2022,88911,85101,CASHEW ROAD,679637,Others,Hospital,Social Services (Without Accommodations),Infant Care Services; Child Minding Services F...
7,201711911W,MAPLEBEAR LEARNING GARDEN PTE. LTD.,None,LOCAL COMPANY,LIVE COMPANY,01-05-2017,88911,88912,BRADDELL ROAD,579713,Others,Hospital,Social Services (Without Accommodations),Infant Care Services; Child Minding Services F...
8,201540131W,4HANDS DENTAL ASSISTING TRAINING SCHOOL PTE. LTD.,None,LOCAL COMPANY,LIVE COMPANY,10-11-2015,88991,na,JURONG WEST STREET 64,641684,Others,Hospital,Social Services (Without Accommodations),Job Training And Vocational Rehabilitation Ser...
9,202337418G,OUT OF THE BOX ACADEMY (CLEMENTI) PTE. LTD.,None,LOCAL COMPANY,LIVE COMPANY,18-09-2023,88912,85509,CLEMENTI AVENUE 3,120433,Others,Hospital,Social Services (Without Accommodations),Student Care Services; Child Minding Services ...


In [22]:
# IMPROVED SCRAPER - With comprehensive extraction from Silver_2
from apify_client import ApifyClient
import pandas as pd
import time

client = ApifyClient("apify_api_0OGDbZtagS7kgb4c9Z8bRNuxAx5Hgi0oFRwE")

# CONFIG
BATCH_SIZE = 5
MAX_CONCURRENCY = 2
MAX_RETRIES = 2

def create_pagefunction_v4() -> str:
    """Enhanced pageFunction with comprehensive data extraction from Silver_2"""
    return """
async function pageFunction(context) {
    const { page, log, request } = context;
    const uen = request?.userData?.uen || '';
    const wait = (ms) => new Promise(resolve => setTimeout(resolve, ms));

    log.info(`🚀 SCRAPING UEN: ${uen} from ${page.url()}`);
    if (!uen) return { status: 'error', uen: null, error: 'Missing UEN' };

    try {
        // Wait for search results
        log.info('⏳ Waiting for search results...');
        await page.waitForSelector('.flex-1.min-w-0', { timeout: 40000 });
        await wait(1500);

        // Find and click company link
        log.info('🔍 Finding company link...');
        const linkFound = await page.evaluate((targetUen) => {
            const uenUpper = targetUen.toUpperCase();
            const allLinks = document.querySelectorAll('a[href*="/company/"]');
            
            for (const link of allLinks) {
                let parent = link.parentElement;
                for (let i = 0; i < 5 && parent; i++) {
                    const parentText = (parent.innerText || parent.textContent || '');
                    if (parentText.toUpperCase().includes(uenUpper)) {
                        link.click();
                        return true;
                    }
                    parent = parent.parentElement;
                }
            }
            if (allLinks.length > 0) { allLinks[0].click(); return true; }
            return false;
        }, uen);

        if (!linkFound) {
            return { status: 'not_found', uen, error: 'No search results' };
        }

        // Wait for company page
        log.info('📄 Navigating to company page...');
        await page.waitForNavigation({ waitUntil: 'domcontentloaded', timeout: 60000 });
        await wait(2000);

        // Extract data with comprehensive phone extraction from Silver_2
        log.info('📊 Extracting data...');
        const data = await page.evaluate(() => {
            const SOCIAL_MEDIA_DOMAINS = ['facebook.com','linkedin.com','instagram.com','tiktok.com','twitter.com','x.com','youtube.com','pinterest.com'];
            
            // ========== EMAIL EXTRACTION ==========
            const emails = [];
            document.querySelectorAll('a[href^="mailto:"]').forEach(a => {
                const email = a.href.replace('mailto:', '').trim();
                if (email && email.includes('@') && !emails.includes(email)) {
                    emails.push(email);
                }
            });
            
            // ========== COMPREHENSIVE PHONE EXTRACTION ==========
            const phones = [];
            
            // Format phone to Singapore standard
            function formatSingaporePhone(text) {
                const digitsOnly = text.replace(/\\D/g, '');
                
                // Singapore number patterns (8 digits)
                if (digitsOnly.length === 8) {
                    return '+65' + digitsOnly;
                }
                // With country code
                if (digitsOnly.length === 10 && digitsOnly.startsWith('65')) {
                    return '+' + digitsOnly;
                }
                if (digitsOnly.length === 11 && digitsOnly.startsWith('65')) {
                    return '+65' + digitsOnly.slice(2);
                }
                // With +
                if (digitsOnly.length >= 10) {
                    for (let i = 0; i <= digitsOnly.length - 10; i++) {
                        if (digitsOnly.slice(i, i+2) === '65' && digitsOnly.length - i >= 10) {
                            return '+' + digitsOnly.slice(i, i+10);
                        }
                    }
                }
                return null;
            }
            
            // Method 1: Extract from tel: links
            document.querySelectorAll('a[href^="tel:"]').forEach(a => {
                const telHref = a.href.replace('tel:', '').trim();
                const formatted = formatSingaporePhone(telHref);
                if (formatted && !phones.includes(formatted)) {
                    phones.push(formatted);
                }
            });
            
            // Method 2: Extract from dt/dd structure
            const companyKeywords = ['company contact', 'business contact', 'office phone', 'main phone', 'business phone', 'company phone', 'contact number', 'phone', 'tel', 'mobile', 'call', 'contact no'];
            
            document.querySelectorAll('dt').forEach(dt => {
                const dtText = dt.textContent.toLowerCase().trim();
                if (companyKeywords.some(kw => dtText.includes(kw))) {
                    const dd = dt.nextElementSibling;
                    if (dd && dd.tagName === 'DD') {
                        const ddText = dd.textContent.trim();
                        const formatted = formatSingaporePhone(ddText);
                        if (formatted && !phones.includes(formatted)) {
                            phones.push(formatted);
                        }
                    }
                }
            });
            
            // Method 3: Look for phone patterns in visible text
            const bodyText = document.body.innerText || document.body.textContent;
            const phonePatterns = [
                /\\b(\\+65[\\s\\-]?)?([689]\\d{3}[\\s\\-]?\\d{4})\\b/g,
                /\\b65[\\s\\-]?([689]\\d{3})[\\s\\-]?(\\d{4})\\b/g,
                /\\b([689]\\d{3})[\\s\\-](\\d{4})\\b/g
            ];
            
            phonePatterns.forEach(pattern => {
                const matches = bodyText.matchAll(pattern);
                for (const match of matches) {
                    const formatted = formatSingaporePhone(match[0]);
                    if (formatted && !phones.includes(formatted)) {
                        phones.push(formatted);
                    }
                }
            });
            
            // ========== WEBSITE EXTRACTION ==========
            const websites = [];
            document.querySelectorAll('a[href^="http"]').forEach(a => {
                const href = a.href.trim().toLowerCase();
                if (!SOCIAL_MEDIA_DOMAINS.some(d => href.includes(d)) && !href.includes('recordowl') && !href.includes('apify')) {
                    if (href.match(/\\.(com|sg|net|org|co)/)) {
                        websites.push(a.href);
                    }
                }
            });
            
            // ========== SOCIAL MEDIA EXTRACTION ==========
            const facebook = [];
            const linkedin = [];
            const instagram = [];
            const tiktok = [];
            
            document.querySelectorAll('a[href*="facebook.com"]').forEach(a => {
                if (!facebook.includes(a.href)) facebook.push(a.href);
            });
            document.querySelectorAll('a[href*="linkedin.com"]').forEach(a => {
                if (!linkedin.includes(a.href)) linkedin.push(a.href);
            });
            document.querySelectorAll('a[href*="instagram.com"]').forEach(a => {
                if (!instagram.includes(a.href)) instagram.push(a.href);
            });
            document.querySelectorAll('a[href*="tiktok.com"]').forEach(a => {
                if (!tiktok.includes(a.href)) tiktok.push(a.href);
            });
            
            // ========== ADDRESS EXTRACTION ==========
            let address = null;
            const labelCandidates = ['registered address', 'registered office address', 'address', 'principal place of business'];
            
            document.querySelectorAll('dt').forEach(dt => {
                const dtText = dt.textContent.toLowerCase().trim();
                if (labelCandidates.some(label => dtText.includes(label))) {
                    const dd = dt.nextElementSibling;
                    if (dd && dd.tagName === 'DD') {
                        address = dd.textContent.trim();
                    }
                }
            });

            return {
                emails: emails.length ? emails : null,
                phones: phones.length ? phones : null,
                website: websites.length ? websites[0] : null,
                facebook: facebook.length ? facebook : null,
                linkedin: linkedin.length ? linkedin : null,
                instagram: instagram.length ? instagram : null,
                tiktok: tiktok.length ? tiktok : null,
                address: address
            };
        });

        log.info(`✅ SUCCESS: ${uen} | Phones: ${data.phones ? data.phones.length : 0} | Emails: ${data.emails ? data.emails.length : 0}`);
        return { status: 'success', uen, url: page.url(), ...data };

    } catch (err) {
        log.error(`❌ ERROR for ${uen}: ${err.message}`);
        return { status: 'error', uen, error: err.message };
    }
}
"""

def run_scraper(client, uens):
    start_urls = [{"url": f"https://recordowl.com/search?name={uen}", "userData": {"uen": uen}} for uen in uens]
    
    print(f"\n  📋 Start URLs created:")
    for i, item in enumerate(start_urls, 1):
        print(f"      {i}. {item['url']}")
    
    run_input = {
        "startUrls": start_urls,
        "useChrome": True,
        "headless": True,
        "stealth": True,
        "pageFunction": create_pagefunction_v4(),
        "maxRequestRetries": 3,
        "maxRequestsPerCrawl": len(start_urls),
        "maxConcurrency": MAX_CONCURRENCY,
        "pageLoadTimeoutSecs": 120,
        "pageFunctionTimeoutSecs": 360,
        "waitUntil": ["domcontentloaded"],
        "proxyConfiguration": {"useApifyProxy": True, "apifyProxyGroups": ["RESIDENTIAL"]},
    }

    print(f"\n  🚀 Launching Apify actor with {MAX_CONCURRENCY} concurrent browsers...")
    run = client.actor("apify/puppeteer-scraper").call(run_input=run_input)
    print(f"  ⏳ Run ID: {run['id']}")
    
    run_client = client.run(run["id"])
    run_info = run_client.wait_for_finish()
    
    print(f"  📊 Status: {run_info.get('status')}")
    
    if run_info.get("status") == "SUCCEEDED" and "defaultDatasetId" in run:
        time.sleep(2)
        dataset = client.dataset(run["defaultDatasetId"])
        items = list(dataset.iterate_items())
        print(f"  ✅ Retrieved {len(items)}/{len(uens)} results")
        return items, None
    
    return [], "Scraping failed"


# Execute scraper
print("="*70)
print("🎯 APIFY SCRAPER - V4 (Comprehensive Extraction)")
print("="*70)
print(f"📊 Config: {MAX_CONCURRENCY} concurrent, {BATCH_SIZE} UENs per batch")

all_results = []
total_rows = len(acra_data_filtered_by_industry)

for batch_idx in range(0, total_rows, BATCH_SIZE):
    batch = acra_data_filtered_by_industry.iloc[batch_idx:batch_idx + BATCH_SIZE]
    uens = [str(row['UEN']).strip() for _, row in batch.iterrows()]
    
    print(f"\n{'─'*70}")
    print(f"📦 Batch {(batch_idx//BATCH_SIZE)+1} - UENs: {', '.join(uens)}")
    
    items, error = run_scraper(client, uens)
    
    if error:
        print(f"  ❌ Batch error: {error}")
        for uen in uens:
            all_results.append({
                "UEN": uen, "Status": "error", "Error": error,
                **{k: None for k in ['Emails','Phones','Website','Facebook','LinkedIn','Instagram','TikTok','address','RecordOwl_Link']}
            })
        continue
    
    # Map results
    uen_map = {item.get('uen'): item for item in items if item.get('uen')}
    
    for uen in uens:
        item = uen_map.get(uen)
        if not item:
            print(f"    ⚠️  {uen}: Not found in results")
            all_results.append({
                "UEN": uen, "Status": "missing", "Error": "No data",
                **{k: None for k in ['Emails','Phones','Website','Facebook','LinkedIn','Instagram','TikTok','address','RecordOwl_Link']}
            })
        else:
            status = item.get('status', 'error')
            if status == 'success':
                phones = len(item.get('phones', [])) if item.get('phones') else 0
                emails = len(item.get('emails', [])) if item.get('emails') else 0
                website = 'Yes' if item.get('website') else 'No'
                print(f"    ✅ {uen}: {phones} phones, {emails} emails, Website: {website}")
            else:
                print(f"    ❌ {uen}: {status} - {item.get('error', 'Unknown')}")
            
            all_results.append({
                'UEN': uen,
                'Status': status,
                'Error': item.get('error'),
                'Emails': item.get('emails'),
                'Phones': item.get('phones'),
                'Website': item.get('website'),
                'Facebook': item.get('facebook'),
                'LinkedIn': item.get('linkedin'),
                'Instagram': item.get('instagram'),
                'TikTok': item.get('tiktok'),
                'address': item.get('address'),
                'RecordOwl_Link': item.get('url')
            })
    
    time.sleep(10)

# Create DataFrame
New_Fresh_Leads = pd.DataFrame(all_results)

print(f"\n{'='*70}")
print(f"✅ COMPLETE")
print(f"{'='*70}")
print(f"📊 Total: {len(New_Fresh_Leads)}")
print(f"✅ Success: {(New_Fresh_Leads['Status']=='success').sum()}")
print(f"❌ Failed: {(New_Fresh_Leads['Status']=='error').sum()}")
print(f"⚠️  Missing: {(New_Fresh_Leads['Status']=='missing').sum()}")
print(f"📞 Phones found: {New_Fresh_Leads['Phones'].notna().sum()}")
print(f"📧 Emails found: {New_Fresh_Leads['Emails'].notna().sum()}")
print(f"🌐 Websites found: {New_Fresh_Leads['Website'].notna().sum()}")
print(f"📘 Facebook found: {New_Fresh_Leads['Facebook'].notna().sum()}")
print(f"📸 Instagram found: {New_Fresh_Leads['Instagram'].notna().sum()}")
print(f"{'='*70}")

New_Fresh_Leads.head(10)

🎯 APIFY SCRAPER - V4 (Comprehensive Extraction)
📊 Config: 2 concurrent, 5 UENs per batch

──────────────────────────────────────────────────────────────────────
📦 Batch 1 - UENs: 53431824W, 202344030R, T15LL1885G, 53200915X, 201733719E

  📋 Start URLs created:
      1. https://recordowl.com/search?name=53431824W
      2. https://recordowl.com/search?name=202344030R
      3. https://recordowl.com/search?name=T15LL1885G
      4. https://recordowl.com/search?name=53200915X
      5. https://recordowl.com/search?name=201733719E

  🚀 Launching Apify actor with 2 concurrent browsers...


[apify.puppeteer-scraper runId:jVNlEqozqHXboxddo] -> Status: RUNNING, Message: 
[apify.puppeteer-scraper runId:jVNlEqozqHXboxddo] -> 2025-11-13T07:17:12.276Z ACTOR: Pulling container image of build g6G5r98rF5fM6ecm3 from registry.
[apify.puppeteer-scraper runId:jVNlEqozqHXboxddo] -> 2025-11-13T07:17:12.279Z ACTOR: Creating container.
[apify.puppeteer-scraper runId:jVNlEqozqHXboxddo] -> 2025-11-13T07:17:12.324Z ACTOR: Starting container.
[apify.puppeteer-scraper runId:jVNlEqozqHXboxddo] -> 2025-11-13T07:17:12.500Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c ./start_xvfb_and_run_cmd.sh && npm run start:prod --silent
[apify.puppeteer-scraper runId:jVNlEqozqHXboxddo] -> 2025-11-13T07:17:13.886Z INFO  System info {"apifyVersion":"3.4.4","apifyClientVersion":"2.16.0","crawleeVersion":"3.14.1","osType":"Linux","nodeVersion":"v22.19.0"}
[apify.puppeteer-scraper runId:jVNlEqozqHXboxddo] -> 2025-11-13T07:17:14.063Z INFO  Configuring Puppeteer Scraper

  ⏳ Run ID: jVNlEqozqHXboxddo
  📊 Status: SUCCEEDED
  ✅ Retrieved 5/5 results
    ✅ 53431824W: 0 phones, 0 emails, Website: Yes
    ✅ 202344030R: 1 phones, 0 emails, Website: No
    ✅ T15LL1885G: 1 phones, 0 emails, Website: Yes
    ✅ 53200915X: 0 phones, 0 emails, Website: Yes
    ✅ 201733719E: 1 phones, 0 emails, Website: Yes

──────────────────────────────────────────────────────────────────────
📦 Batch 2 - UENs: 53227394W, 202209857Z, 201711911W, 201540131W, 202337418G

  📋 Start URLs created:
      1. https://recordowl.com/search?name=53227394W
      2. https://recordowl.com/search?name=202209857Z
      3. https://recordowl.com/search?name=201711911W
      4. https://recordowl.com/search?name=201540131W
      5. https://recordowl.com/search?name=202337418G

  🚀 Launching Apify actor with 2 concurrent browsers...


[apify.puppeteer-scraper runId:EKkjMzmqZ7lbdm83H] -> Status: RUNNING, Message: Starting the crawler.
[apify.puppeteer-scraper runId:EKkjMzmqZ7lbdm83H] -> 2025-11-13T07:18:27.426Z ACTOR: Pulling container image of build g6G5r98rF5fM6ecm3 from registry.
[apify.puppeteer-scraper runId:EKkjMzmqZ7lbdm83H] -> 2025-11-13T07:18:27.428Z ACTOR: Creating container.
[apify.puppeteer-scraper runId:EKkjMzmqZ7lbdm83H] -> 2025-11-13T07:18:27.487Z ACTOR: Starting container.
[apify.puppeteer-scraper runId:EKkjMzmqZ7lbdm83H] -> 2025-11-13T07:18:27.649Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c ./start_xvfb_and_run_cmd.sh && npm run start:prod --silent
[apify.puppeteer-scraper runId:EKkjMzmqZ7lbdm83H] -> 2025-11-13T07:18:28.381Z INFO  System info {"apifyVersion":"3.4.4","apifyClientVersion":"2.16.0","crawleeVersion":"3.14.1","osType":"Linux","nodeVersion":"v22.19.0"}
[apify.puppeteer-scraper runId:EKkjMzmqZ7lbdm83H] -> 2025-11-13T07:18:29.170Z INFO  Configur

  ⏳ Run ID: EKkjMzmqZ7lbdm83H
  📊 Status: SUCCEEDED
  ✅ Retrieved 5/5 results
    ✅ 53227394W: 1 phones, 0 emails, Website: No
    ✅ 202209857Z: 1 phones, 0 emails, Website: Yes
    ✅ 201711911W: 1 phones, 1 emails, Website: Yes
    ✅ 201540131W: 1 phones, 1 emails, Website: Yes
    ✅ 202337418G: 1 phones, 1 emails, Website: No

✅ COMPLETE
📊 Total: 10
✅ Success: 10
❌ Failed: 0
⚠️  Missing: 0
📞 Phones found: 8
📧 Emails found: 3
🌐 Websites found: 7
📘 Facebook found: 4
📸 Instagram found: 4


,UEN,Status,Error,Emails,Phones,Website,Facebook,LinkedIn,Instagram,TikTok,address,RecordOwl_Link
0,53431824W,success,None,None,None,https://www.tutorsville.sg/,None,None,None,None,230 COMPASSVALE WALK #10-430 SINGAPORE 540230,https://recordowl.com/company/tutorsville-sg
1,202344030R,success,None,None,[+6591943237],None,None,None,None,None,87 BEACH ROAD #04-03 CHYE SING BUILDING 189695,https://recordowl.com/company/chem-affinity-le...
2,T15LL1885G,success,None,None,[+6568176157],https://edureachsg.com/,[https://www.facebook.com/EduReachServices],None,None,None,201E TAMPINES STREET 23 #01-122 SINGAPORE 527201,https://recordowl.com/company/edureach-service...
3,53200915X,success,None,None,None,https://www.thinkarts.co.in/,None,None,[https://www.instagram.com/thinkartsforchildre...,None,YARROW GARDENS SINGAPORE 455021,https://recordowl.com/company/think-arts
4,201733719E,success,None,None,[+6586860777],https://jusinfants.com/,None,None,[https://www.instagram.com/jusinfantsplayhouse/],None,47 KALLANG PUDDING ROAD #01-01 THE CRESCENT @ ...,https://recordowl.com/company/jus-infants-macp...
5,53227394W,success,None,None,[+6598524810],None,None,None,None,None,555 ANG MO KIO AVENUE 10 #01-1958 CHENG SAN PL...,https://recordowl.com/company/maths-tablet
6,202209857Z,success,None,None,[+6598193093],http://www.yorkearlyyears.com/,None,None,None,None,54 CASHEW ROAD #02-02 CASHEW PARK CONDOMINIUM ...,https://recordowl.com/company/york-education-p...
7,201711911W,success,None,[principal-rochester@maplebear.sg],[+6562521488],https://maplebear.sg/contact-us,[https://www.facebook.com/maplebearsg],None,[https://www.instagram.com/maplebearsingapore/...,None,351 BRADDELL ROAD #03-01 351 ON BRADDELL SINGA...,https://recordowl.com/company/maplebear-learni...
8,201540131W,success,None,[sg_4hands@outlook.com],[+6594203111],https://4handsda.com/,[https://www.facebook.com/4HandsDA/],None,None,None,684A JURONG WEST STREET 64 #17-103 SINGAPORE 6...,https://recordowl.com/company/4hands-dental-as...
9,202337418G,success,None,[enquiries@outoftheboxacademy.com],[+6580664284],None,"[https://www.facebook.com/otbstudentcare, http...",None,[https://www.instagram.com/otbstudentcare/],None,433 CLEMENTI AVENUE 3 #01-258 120433,https://recordowl.com/company/out-of-the-box-a...


In [23]:
# Pre-compile patterns for speed
POSTAL_RE = re.compile(r"(?:\bSingapore\b\s*)?(?P<postal>\d{6})(?!\d)", re.IGNORECASE)
UNIT_RES = [
    re.compile(r"#\s*[A-Za-z0-9]{1,4}\s*[-–]\s*[A-Za-z0-9]{1,4}", re.IGNORECASE),
    re.compile(r"\bunit\s*[#:]?\s*[A-Za-z0-9]{1,4}\s*[-–]\s*[A-Za-z0-9]{1,4}\b", re.IGNORECASE),
    re.compile(r"\bunit\s*[#:]?\s*[A-Za-z0-9]{1,5}\b", re.IGNORECASE),
]

def normalize_spaces(text: str) -> str:
    text = re.sub(r"[\n\r\t]+", " ", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip(" ,;|/")

def extract_postal(text: str) -> tuple[str, str | None]:
    if not text:
        return text, None
    matches = list(POSTAL_RE.finditer(text))
    if matches:
        m = matches[-1]
        postal = m.group("postal")
        start, end = m.span()
        cleaned = text[:start] + text[end:]
        cleaned = re.sub(r"\bSingapore\b", "", cleaned, flags=re.IGNORECASE)
        return normalize_spaces(cleaned), postal
    return normalize_spaces(text), None

def extract_unit(text: str) -> tuple[str, str | None]:
    if not text:
        return text, None
    for rx in UNIT_RES:
        m = rx.search(text)
        if m:
            unit_raw = m.group(0)
            cleaned = normalize_spaces(text[:m.start()] + text[m.end():])
            unit_digits = re.sub(r"^unit\s*[#:]?\s*", "", unit_raw, flags=re.IGNORECASE)
            unit_digits = normalize_spaces(unit_digits)
            unit_digits = unit_digits.replace(' – ', '-').replace('–', '-').replace(' ', '')
            unit_digits = unit_digits.lstrip('#')
            return cleaned, unit_digits
    return normalize_spaces(text), None

def clean_street(text: str) -> str | None:
    if not text:
        return None
    text = normalize_spaces(text)
    text = re.sub(r"\s*,\s*", ", ", text)
    return text if text.isupper() else text.title()

def split_address_sg(address: str) -> dict:
    if not isinstance(address, str) or not address.strip():
        return {"street": None, "unit": None, "postal_code": None, "address_clean": None}
    raw = normalize_spaces(address)
    without_postal, postal = extract_postal(raw)
    without_unit, unit = extract_unit(without_postal)
    without_unit = normalize_spaces(re.sub(r"\bSingapore\b", "", without_unit, flags=re.IGNORECASE))
    street = clean_street(without_unit)
    address_clean = normalize_spaces(" ".join(x for x in [street or "", unit or "", f"Singapore {postal}" if postal else ""] if x))
    return {"street": street, "unit": unit, "postal_code": postal, "address_clean": address_clean}

# Apply to current result DF -> create a new dataframe with clean components
if 'address' not in New_Fresh_Leads.columns:
    raise ValueError("Column 'address' not found in New_Fresh_Leads. Run the scraping cell first.")

parsed_df = pd.DataFrame(list(New_Fresh_Leads["address"].apply(split_address_sg)))

# New DataFrame with clean address fields and without raw 'address'
Cleaned_New_Fresh_Leads = New_Fresh_Leads.copy()
if 'address' in Cleaned_New_Fresh_Leads.columns:
    Cleaned_New_Fresh_Leads = Cleaned_New_Fresh_Leads.drop(columns=['address'])
Cleaned_New_Fresh_Leads["operational_street"] = parsed_df["street"]
Cleaned_New_Fresh_Leads["operational_unit"] = parsed_df["unit"]
Cleaned_New_Fresh_Leads["operational_postal_code"] = parsed_df["postal_code"]
Cleaned_New_Fresh_Leads["operational_address"] = parsed_df["address_clean"]

# Save full result to a new DataFrame and display all columns
New_Fresh_Leads_Operational = Cleaned_New_Fresh_Leads.copy()

In [24]:
New_Fresh_Leads_Operational

,UEN,Status,Error,Emails,Phones,Website,Facebook,LinkedIn,Instagram,TikTok,RecordOwl_Link,operational_street,operational_unit,operational_postal_code,operational_address
0,53431824W,success,None,None,None,https://www.tutorsville.sg/,None,None,None,None,https://recordowl.com/company/tutorsville-sg,230 COMPASSVALE WALK,10-430,540230,230 COMPASSVALE WALK 10-430 Singapore 540230
1,202344030R,success,None,None,[+6591943237],None,None,None,None,None,https://recordowl.com/company/chem-affinity-le...,87 BEACH ROAD CHYE SING BUILDING,04-03,189695,87 BEACH ROAD CHYE SING BUILDING 04-03 Singapo...
2,T15LL1885G,success,None,None,[+6568176157],https://edureachsg.com/,[https://www.facebook.com/EduReachServices],None,None,None,https://recordowl.com/company/edureach-service...,201E TAMPINES STREET 23,01-122,527201,201E TAMPINES STREET 23 01-122 Singapore 527201
3,53200915X,success,None,None,None,https://www.thinkarts.co.in/,None,None,[https://www.instagram.com/thinkartsforchildre...,None,https://recordowl.com/company/think-arts,YARROW GARDENS,None,455021,YARROW GARDENS Singapore 455021
4,201733719E,success,None,None,[+6586860777],https://jusinfants.com/,None,None,[https://www.instagram.com/jusinfantsplayhouse/],None,https://recordowl.com/company/jus-infants-macp...,47 KALLANG PUDDING ROAD THE CRESCENT @ KALLANG,01-01,349318,47 KALLANG PUDDING ROAD THE CRESCENT @ KALLANG...
5,53227394W,success,None,None,[+6598524810],None,None,None,None,None,https://recordowl.com/company/maths-tablet,555 ANG MO KIO AVENUE 10 CHENG SAN PLACE,01-1958,560555,555 ANG MO KIO AVENUE 10 CHENG SAN PLACE 01-19...
6,202209857Z,success,None,None,[+6598193093],http://www.yorkearlyyears.com/,None,None,None,None,https://recordowl.com/company/york-education-p...,54 CASHEW ROAD CASHEW PARK CONDOMINIUM,02-02,679637,54 CASHEW ROAD CASHEW PARK CONDOMINIUM 02-02 S...
7,201711911W,success,None,[principal-rochester@maplebear.sg],[+6562521488],https://maplebear.sg/contact-us,[https://www.facebook.com/maplebearsg],None,[https://www.instagram.com/maplebearsingapore/...,None,https://recordowl.com/company/maplebear-learni...,351 BRADDELL ROAD 351 ON BRADDELL,03-01,579713,351 BRADDELL ROAD 351 ON BRADDELL 03-01 Singap...
8,201540131W,success,None,[sg_4hands@outlook.com],[+6594203111],https://4handsda.com/,[https://www.facebook.com/4HandsDA/],None,None,None,https://recordowl.com/company/4hands-dental-as...,684A JURONG WEST STREET 64,17-103,641684,684A JURONG WEST STREET 64 17-103 Singapore 64...
9,202337418G,success,None,[enquiries@outoftheboxacademy.com],[+6580664284],None,"[https://www.facebook.com/otbstudentcare, http...",None,[https://www.instagram.com/otbstudentcare/],None,https://recordowl.com/company/out-of-the-box-a...,433 CLEMENTI AVENUE 3,01-258,120433,433 CLEMENTI AVENUE 3 01-258 Singapore 120433
